#  huggingface-transformers(BERT)

本書の最後に、最近の自然言語処理技術を利用するためのフレームワークである Transformer を紹介します。また、Transformer で利用する言語モデルとして BERT を取り上げます。

**なお、本書のp.24で説明した通り、requirements.txt を利用してインストールすると、必要なパッケージはすでに導入されています。
requirements.txt を利用せずにインストールする場合は `!pip install` コマンドを使って個別にインストールすることになります。その場合、バージョン指定をしなければ、最新のライブラリがインストールされます。特に transformers は仕様が変わっている可能性があるため、本書の記載どおりでは動作しないコードがあるかもしれません。筆者側で補足できた範囲で、このGitHub レポジトリでサポートしていきます。**

## ディープラーニングと自然言語処理


以下、本書の最後に、huggingface-transoformers （ BERT）を利用したテキスト処理の技法を紹介しましょう（GiNZA を紹介した章で述べたように、GiNZA で Transformer にもとづく学習モデルを読み込むという方法もあります）。

ところで、ディープラーニングは計算量が多いため、標準的なパソコンでは処理に非常に時間がかかることがあります。そこでパソコンにグラフィックボードという画像処理のハードウェア(**GPU**)を追加することで、処理の多くをGPUに分散させ、負荷の軽減と高速化をはかることができます。


とはいえ、GPUの導入とドライバのインストールは簡単ではありません。
幸い、自身のパソコンにGPUがない場合でも、 huggingface-transformers を試す方法があります。Google Collabaratory [^gcollab] という無償のWEBサービスを使うことです（以下 Colab と表記します）。Colab は、ここまで利用してきた Jupyter (Jupyter Labo) とほぼ同じ感覚で利用することができます。


Google Coraboratory での作業方法については、巻末の付録の付録にも記していますが、ここでも改めて解説いたします。



```
## Google Colaboratory における Mecabのインストール
!apt install mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3
!pip install fugashi ipadic
!pip install torch
```


`!pip install transformers==4.12.0`

なお、Colabではなく自身のマシンで実行する場合は torch (PyTorch) もインストールしてください。
筆者が Colab 上で作業したファイルを Google Colab に公開していますので、その URL をサポートサイトで確認してください。

<https://colab.research.google.com/drive/1E13hvgiCmh_eZtvnnZHh59s3_FTR4I9J>


このノートブックを自身のドライブにコピーした上で、試してみて下さい。


## transformers によるトークン化

transformers で日本語を扱うには、日本語トークンにもとづく事前学習モデルを導入する必要があります。

transformers では東北大学の自然言語処理研究室が開発したモデルを利用することできます。このモデルは、日本語ウキペディアをデータとして学習されたモデルになります。ここでは'bert-base-japanese-whole-word-masking'を利用しますが、2021年により大きなモデル 'bert-large-japanese'が公開されています。ただし、大きなモデルを使う場合、GPU のメモリが足りず、RuntimeError: CUDA error: out of memory というエラーで作業が進まなくなることがあるので、注意してください。

なお `AutoTokenizer` モジュールは、指定されたモデルのトークナイザーに適切な設定を行ってくれます。


In [1]:
import torch
from transformers import AutoTokenizer
## ここでは'bert-base-japanese-whole-word-masking'を利用しますが、2021年により大きなモデル 'bert-large-japanese'が公開されています。
## ただし、大きなモデルを使う場合、GPU のメモリが足りず、RuntimeError: CUDA error: out of memory 
## というエラーで作業が進まなくなることがあるので、注意してください。
japanese_model = ('cl-tohoku/bert-base-japanese-whole-word-masking')
# 
tokenizer = AutoTokenizer.from_pretrained(japanese_model)
res = tokenizer.tokenize('最近の自然言語処理の主流はディープラーニングだ。')
# print(res)
ids = tokenizer.convert_tokens_to_ids(res)
tokens = tokenizer.convert_ids_to_tokens(ids)
## 単語IDを確認
print(ids)
## 対応するトークン（形態素、文字など）を確認
print(tokens)

[5233, 5, 1757, 1882, 2762, 5, 5770, 9, 14872, 422, 1581, 75, 8]
['最近', 'の', '自然', '言語', '処理', 'の', '主流', 'は', 'ディープ', '##ラー', '##ニング', 'だ', '。']



## 単語ID

トークン化について説明しましたが、単語はそのままで処理されるわけではありません。内部でトークンには一意のID（番号）が割り当てられ、それが入力となります。

In [2]:
print(tokenizer('彼は蕎麦を食べた。'))

{'input_ids': [2, 306, 9, 26724, 11, 2949, 10, 8, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}


## トークン穴埋め問題


transformers を使って文章をトークンに分割できるようになりました。次に、分割した結果を言語モデルに適用してみましょう。
transformers では、空白を推定するタスクを行うのに **AutoModelForMaskedLM** クラスに、言語ごとに用意されたモデルをアタッチします。以前は言語モデルごとにクラス名が異なってました。
たとえば、BERT 日本語モデルを指定するという意味で、BertForMaskedLM クラスで日本語モデルを読み込んでいました。


In [3]:
from transformers import AutoConfig, AutoModelForMaskedLM
masked_model = AutoModelForMaskedLM.from_pretrained(japanese_model)
## GPUを搭載しているのであれば、GPUのメモリを使う
masked_model = masked_model.cuda()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
text = '今日は[MASK]で勉強した。'
tokens = tokenizer.tokenize(text)
print(tokens)

['今日', 'は', '[MASK]', 'で', '勉強', 'し', 'た', '。']



トークン列を符号化して、モデルへの入力とします。

In [5]:
encoded_text = tokenizer.encode(text, return_tensors='pt')
encoded_text = encoded_text.cuda()
with torch.no_grad():
    output = masked_model(input_ids=encoded_text)
    scores = output.logits

出力の `scores` は 3 次元の配列になっています。

In [6]:
print(f'socresのサイズ：{scores.size()}')

socresのサイズ：torch.Size([1, 10, 32000])


In [7]:
print(f'各トークンのID：{encoded_text[0].tolist()}')

各トークンのID：[2, 3246, 9, 4, 12, 8192, 15, 10, 8, 3]


In [8]:
mask_position = encoded_text[0].tolist().index(4)
best_id = scores[0, mask_position].argmax(-1).item()
print(f'ID＝{best_id}')
best_token = tokenizer.convert_ids_to_tokens(best_id)
print(f'トークン={best_token}')

ID＝396
トークン=大学


In [9]:
topK = scores[0, mask_position].topk(10)
print(topK.indices)
tokens  = tokenizer.convert_ids_to_tokens(topK.indices)
print(tokens)

tensor([  396,  1411,  1724,   286, 18949,  1221,  4441,   723,  2184,  1193],
       device='cuda:0')
['大学', 'ここ', 'ニューヨーク', 'アメリカ', 'コロンビア大学', 'そこ', 'ロサンゼルス', 'イギリス', 'パリ', '高校']


### pipeline

huggingface-transformers には自然言語処理でよく行われる処理について、学修済みモデルを簡単に適用できる **pipeline** という仕組みがあります[^pipeline]。以下に一例をあげます。


- 文章穴埋め ('fill-mask')
- 感情分析 ('sentiment-analysis')
- テキスト分類 ('text-classification')
- 固有表現抽出 ('ner')
- 質問応答 ('question-answeri')
- 文章要約 ('summarization')
- 翻訳 ('translation')

[^pipeline]: https://huggingface.co/docs/transformers/main_classes/pipelines


In [10]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model=japanese_model, tokenizer=tokenizer)
print(unmasker('今日は[MASK]で勉強した。'))

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'sequence': '今日 は 大学 で 勉強 し た 。', 'score': 0.06624556332826614, 'token': 396, 'token_str': '大 学'}, {'sequence': '今日 は ここ で 勉強 し た 。', 'score': 0.03463234007358551, 'token': 1411, 'token_str': 'こ こ'}, {'sequence': '今日 は ニューヨーク で 勉強 し た 。', 'score': 0.03233740106225014, 'token': 1724, 'token_str': 'ニ ュ ー ヨ ー ク'}, {'sequence': '今日 は アメリカ で 勉強 し た 。', 'score': 0.02758091315627098, 'token': 286, 'token_str': 'ア メ リ カ'}, {'sequence': '今日 は コロンビア大学 で 勉強 し た 。', 'score': 0.02206672914326191, 'token': 18949, 'token_str': 'コ ロ ン ビ ア 大 学'}]


## huggingface transformers によるテキスト分類

このタスクのクラス名は **AutoModelForSequenceClassification** となります。
ここでジャンル判定の応用として、文章の内容がネガティブなのかポジティブなのか判定する **センチメント分析** を試してみましょう。


In [11]:
from transformers import AutoModelForSequenceClassification, pipeline
## 日本語感情分析用のモデルをロードする
sentiment_model = AutoModelForSequenceClassification.from_pretrained ('daigo/bert-base-japanese-sentiment')
sentiment_analyzer = pipeline("sentiment-analysis", model=sentiment_model, tokenizer=tokenizer)
print(sentiment_analyzer('ロシアとウクライナの戦争はまだ終わらない。'))

[{'label': 'ネガティブ', 'score': 0.7547963261604309}]



ここではテキストをジャンルごとに分類を行うための学習済みモデルを、新たに用意したテキストデータセットでファインチューニングする方法を紹介しましょう



まず分析対象とするテキストデータセットを用意します。ここでは、自然言語処理でベンチマークとしてよく利用される Livedoor ニュースコーパスを利用させてもらいます[^livedoor]。
株式会社ロンウイットのサイトから ldcc-20140209.tar.gz というファイルをダウンロードします。

[^livedoor]: https://www.rondhuit.com/download.html#ldcc

以下では Python の関数を使ってダウンロードと解凍を行っています。

In [12]:
## データセットのダウンロード
# !wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
# !tar xvzf ldcc-20140209.tar.gz
from urllib import request
request.urlretrieve("https://www.rondhuit.com/download/ldcc-20140209.tar.gz", "ldcc-20140209.tar.gz")
## 解凍
import tarfile
with tarfile.open('ldcc-20140209.tar.gz', 'r:gz') as t:
    t.extractall(path='.')

なお、Google Colaboratory を利用している場合は、保存用のフォルダ（ディレクトリ）を用意します。


In [ ]:
## Google Colaboratory で作業する場合
from google.colab import drive 
drive.mount('/content/drive')
!mkdir -p /content/drive/MyDrive
## 作業フォルダを移動
%cd /content/drive/MyDrive
!pwd

In [ ]:
import os
print(os.getcwd())

作業フォルダ text には 10 個のサブフォルダが含まれています。サブフォルダ名を確認してみましょう。

In [15]:
## サブフォルダを確認
#
#categories = [name for name in os.listdir("../livedoor/text") if os.path.isdir("../livedoor/text/" + name)]
# print(categories)

['smax', 'kaden-channel', 'it-life-hack', 'dokujo-tsushin', 'livedoor-homme', 'sports-watch', 'movie-enter', 'peachy', 'topic-news']


In [16]:
import os 
categories = [name for name in os.listdir("text") if os.path.isdir("text/" + name)]
print(categories)

['smax', 'kaden-channel', 'it-life-hack', 'dokujo-tsushin', 'livedoor-homme', 'sports-watch', 'movie-enter', 'peachy', 'topic-news']


Livedoor ファイルには 10 種類のジャンルのファイルがあります。

In [ ]:
from glob import glob
import pandas as pd
categories = ['it-life-hack', 'dokujo-tsushin'] 
datasets = pd.DataFrame(columns=["sentences", "labels"])
for label, cat in enumerate(categories):
    for file in glob(f'text/{cat}/{cat}*'):# for file in glob(f'../livedoor/text/{cat}/{cat}*'):
        ## Google Colaboratory の場合は file in glob(f'text/{cat}/{cat}*'): と変更
        lines = open(file).read().splitlines()
        body = '\n'.join(lines[3:])
        sentences = pd.Series([body, cat], index=datasets.columns)
        datasets = datasets.append(sentences, ignore_index=True)

datasets.head()

テキストのジャンルを表す文字列を数値に変えます。it-life-hack には 0 を、dokujo-tsushin には 1 を対応させます。この対応を辞書として用意し `map()` で lables 列に一括適用します。

In [19]:
cat_id = dict(zip(categories, list(range(len(categories)))))
print(cat_id)
datasets['labels'] = datasets['labels'].map(cat_id) 

{'it-life-hack': 0, 'dokujo-tsushin': 1}


In [20]:
print(datasets['labels'])

0       0
1       0
2       0
3       0
4       0
       ..
1735    1
1736    1
1737    1
1738    1
1739    1
Name: labels, Length: 1740, dtype: int64


さて、transformers を使ってトークンに分割します。

In [21]:
import torch
from transformers import AutoTokenizer
japanese_model = 'cl-tohoku/bert-base-japanese-whole-word-masking'# 'cl-tohoku/bert-large-japanese' # '
tokenizer = AutoTokenizer.from_pretrained(japanese_model)

ここで用意したデータフレームを、訓練用とテスト用に分割します。

In [22]:
import random
random.seed(0)
## ラベル別にindexを取得
label0 = datasets.query('labels==0').index
label1 = datasets.query('labels==1').index
## それぞれから500行を取り出して訓練データとする
rnd0 = random.sample(list(label0), 500)
rnd1 = random.sample(list(label1), 500)
idx = rnd0 + rnd1
train_data = datasets.iloc[idx]
## 残りをテストデータとする
test_data = datasets.drop(index=idx)

In [23]:
## 冒頭の2行を確認
train_data.iloc[:2, :]

,sentences,labels
864,ターガスと言えば、PC関連、特にノートPCを収納しつつ機能性に富むビジネスバッグの定番と言え...,0
394,販促イベントや催事、展示即売会、運動会や体育祭、文化祭、音楽祭といった行事で統一感を出したい...,0


次に、それぞれのデータを huggingface transformers の Trainer クラスに適用できるように加工します。



In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_encodings = tokenizer(train_data['sentences'].tolist(),
                            return_tensors='pt',
                            padding=True, truncation=True,
                            max_length=128).to(device)
test_encodings = tokenizer(test_data['sentences'].tolist(),
                           return_tensors='pt',
                           padding=True, truncation=True,
                           max_length=128).to(device)
train_labels = torch.tensor(train_data['labels'].tolist())
test_labels =  torch.tensor(test_data['labels'].tolist())

さて、これを **Dataset** というクラスのオブジェクトに変換します。このために、クラスを独自に定義します。


In [25]:
class LiveDoor_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

## 実際にデータを変換する
train_dataset = LiveDoor_Dataset(train_encodings, train_labels)
test_dataset = LiveDoor_Dataset(test_encodings, test_labels)

In [26]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(    
    japanese_model,
    num_labels = 2, 
    output_attentions = False, 
    output_hidden_states = False
)

## モデルをGPUに載せる
if torch.cuda.is_available():
    model.cuda()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

TrainingArguments クラスに学習の精度を評価するメソッドを指定するために定義をしておきます。
**評価指標** とは、機械学習やディープラーニングにおいては、予測値あるいは分類の精度を検討するための基準のことです。

- 正解率 (Accuracy)
- 精度 (Precision)
- 検出率 (Recall)
- F 値 (F-measure, F-score, F1 Score)


これらを説明するには **混同行列** を知っておく必要があります。

|メール番号 | スパムか否か|予測結果 |
|-------|---------|--------|
|メール1 |0 | 0|
|メール2 |0 | 1|
|メール3 |0 | 0|
|メール4 |0 | 0| 
|メール5 |1 | 1 |
|メール6 |0 | 0 |
|メール7 |1 | 1|
|メール8 |1 | 1 |
|メール9 |0 | 0 |
|メール10 |1 | 1|




In [27]:
from sklearn.metrics import confusion_matrix
true_label = [0, 0, 0, 1, 0, 1, 1, 0, 1, 0]
pred_label = [0, 0, 0, 1, 0, 1, 1, 1, 0, 1]
cm = confusion_matrix(true_label, pred_label)
print(cm)

[[4 2]
 [1 3]]


具体的には以下のような表になります。


|        |分類結果        |             |
|--------|-------------|--------------|
|        |　スパム(1)と分類 | スパム(0)と分類　|
|実際     |　            |            　|
|スパム(1) |      4      |       2     |
|非スパム(0)|      1      |      3       |



さて、混同行列では、各セルが次の評価に対応します。



|     |分類結果|    |
|--------|-------------|--------------|
|     |　陽性    | 陰性　         |
|実際  |　       |    　         |
| 陽性 | TP 真陽性 |   FN 真陰性|
| 陰性 | FP 偽陽性 |  TF 真陰性 |


- TP (True-Positive) 真陽性： 本当は陽性（スパム）であるメールを、正しく陽性と判定
- TN (True-Negative) 真陰性：本当は陰性（非スパム）を、正しく陰性と判定
- FP (False-Positive) 偽陽性： 本当は陰性であるメールを、誤って陽性と判定
- FN (False-Negative) 偽陰性： 本当は陽性であるメールを、誤って陰性と判定



それぞれの個数は次のように求められます。


In [28]:
tp, fn, fp, tn = cm.ravel()
print((tp, fn, fp, tn))

(4, 2, 1, 3)


In [29]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print(f'正解率: {accuracy_score(true_label, pred_label)}')
print(f'適合率: {precision_score(true_label, pred_label)}')
print(f'感度: {recall_score(true_label, pred_label)}')
print(f'F 値: {f1_score(true_label, pred_label)}')

正解率: 0.7
適合率: 0.6
感度: 0.75
F 値: 0.6666666666666665


なお、正解率、適応率、F値を一度に求められる `precision_recall_fscore_support` というメソッドもあります。

In [30]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
## 4 つの指標を計算する関数を定義
def cal_4metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

データまた評価指標の用意ができたので、Trainer クラスを使って学習を行います
ここでは、パソコンにあまり負荷をかけず、早期に学習が終了することを優先した設定としています。

In [31]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results', 
    num_train_epochs=1, 
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=64, 
    warmup_steps=500,  
    weight_decay=0.01, 
    save_total_limit=1, 
    load_best_model_at_end=True,
    dataloader_pin_memory=False, 
    evaluation_strategy="steps",
    logging_steps=50,
    logging_dir='./logs'
)

trainer = Trainer(
    model=model, 
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset, 
    compute_metrics=cal_4metrics 
)

trainer.train()

***** Running training *****
  Num examples = 1000
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 63
/tmp/ipykernel_80864/291164983.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,0.630400,0.419805,0.913514,0.913514,0.913514,0.913514


***** Running Evaluation *****
  Num examples = 740
  Batch size = 64
/tmp/ipykernel_80864/291164983.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=63, training_loss=0.5893490882146926, metrics={'train_runtime': 19.621, 'train_samples_per_second': 50.966, 'train_steps_per_second': 3.211, 'total_flos': 65777763840000.0, 'train_loss': 0.5893490882146926, 'epoch': 1.0})

取り除けておいたテストデータを評価します。

In [32]:
print(trainer.evaluate(eval_dataset=test_dataset))

***** Running Evaluation *****
  Num examples = 740
  Batch size = 64
/tmp/ipykernel_80864/291164983.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'eval_loss': 0.3153570294380188, 'eval_accuracy': 0.9175675675675675, 'eval_f1': 0.9175662127390699, 'eval_precision': 0.9175950208560095, 'eval_recall': 0.9175675675675675, 'eval_runtime': 3.3668, 'eval_samples_per_second': 219.794, 'eval_steps_per_second': 3.564, 'epoch': 1.0}


上記のテストデータの評価結果を見ると、accuracy が 0.92 F1 score が 0.92 となりました。

ファインチューニングしたモデルは以下のように保存することができます。

In [33]:
model_directory = './LiveDoor_model'
# %%script false --no-raise-error
tokenizer.save_pretrained(model_directory)
model.save_pretrained(model_directory)

tokenizer config file saved in ./LiveDoor_model/tokenizer_config.json
Special tokens file saved in ./LiveDoor_model/special_tokens_map.json
Configuration saved in ./LiveDoor_model/config.json
Model weights saved in ./LiveDoor_model/pytorch_model.bin


あるいは同じことになりますが、以下のようにしても保存できます

In [ ]:
model_directory = './LiveDoor_model'
tokenizer.save_model(model_directory)

保存したモデルを読み込む場合には以下のようにします。


In [34]:
from transformers import AutoModelForSequenceClassification
model_directory = './LiveDoor_model'
model2 = AutoModelForSequenceClassification.from_pretrained(model_directory)    

loading configuration file ./LiveDoor_model/config.json
Model config BertConfig {
  "_name_or_path": "cl-tohoku/bert-base-japanese-whole-word-masking",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "tokenizer_class": "BertJapaneseTokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading weights file ./LiveDoor_model/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All th

## まとめ

以上、 huggingface-transformers による自然言語処理の実行例を示しました。
最初にも述べたように、ディープラーニングに基づくライブラリは更新が早く、現在のバージョンでは動作したコードであっても、しばらく後には期待通りの出力が得られないということが多々あります。
そのため、利用するつど最新バージョンにおける関数の定義などを確認する必要があります。

その一方で、ディープラーニングに基づく最新の自然言語処理技術を反映した huggingface-transformers はしばらくの間、デファクトスタンダードの地位を維持すると予想されます。
関数などの仕様の変更は続くと思われますが、その考え方や処理の流れが大きく変化することは当面ないかもしれません。

